# Experiment 049: Manual OOD Solvent Handling

**Hypothesis**: Manually identify high-error solvents and use simpler features (Spange only) for them.

**Based on evaluator's suggestion**: The cosine similarity approach in exp_048 failed because all solvents had similarity >0.99. Instead, we manually identify the high-error solvents from exp_048 analysis:
- HFIP: 0.038 (4.3x mean error)
- Water.Ethanol: 0.028 (3.2x mean error)
- Acetonitrile.Acetic Acid: 0.022 (2.5x mean error)
- TFE: 0.015 (1.7x mean error)

**Implementation**:
- Model A: Full features (Spange + DRFP) with GP + MLP + LGBM ensemble for normal solvents
- Model B: Simple features (Spange only) with MLP + LGBM for high-error solvents
- Hard-coded list of high-error solvents

In [ ]:
import sys
sys.path.insert(0, '/home/code')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Load data
from utils import load_data, generate_leave_one_out_splits, generate_leave_one_ramp_out_splits

print("Loading data...")
X, Y = load_data()
print(f"X shape: {X.shape}, Y shape: {Y.shape}")
print(f"Unique solvents: {X['SOLVENT NAME'].nunique()}")
print(f"Target columns: {Y.columns.tolist()}")

In [ ]:
# Define high-error solvents based on exp_048 analysis
# These are the solvents that consistently have high errors across experiments
HIGH_ERROR_SOLVENTS = [
    '1,1,1,3,3,3-Hexafluoropropan-2-ol',  # HFIP - 0.038 (4.3x mean error)
    '2,2,2-Trifluoroethanol',  # TFE - 0.015 (1.7x mean error)
    'Cyclohexane',  # Biggest outlier by distance from cluster center (4.63)
]

# Also check for mixture solvents that might be problematic
HIGH_ERROR_MIXTURES = [
    'Acetonitrile.Acetic Acid',  # 0.022 (2.5x mean error)
    'Water.Ethanol',  # 0.028 (3.2x mean error)
    'Water.2,2,2-Trifluoroethanol',  # Contains TFE
]

print(f"High-error single solvents: {HIGH_ERROR_SOLVENTS}")
print(f"High-error mixtures: {HIGH_ERROR_MIXTURES}")

# Check which solvents are in the data
all_solvents = X['SOLVENT NAME'].unique()
print(f"\nSolvents in data that match high-error list:")
for s in HIGH_ERROR_SOLVENTS + HIGH_ERROR_MIXTURES:
    if s in all_solvents:
        count = (X['SOLVENT NAME'] == s).sum()
        print(f"  {s}: {count} samples")

In [ ]:
# Feature extraction functions
def get_spange_features(X_data):
    """Extract Spange descriptors (13 features)"""
    spange_cols = ['SP', 'SdP', 'SA', 'SB', 'Dipolarity', 'Polarizability', 
                   'Acidity', 'Basicity', 'Viscosity', 'Surface Tension', 
                   'Refractive Index', 'Density', 'Molar Volume']
    return X_data[spange_cols].values

def get_drfp_features(X_data):
    """Extract DRFP features (high-variance only)"""
    drfp_cols = [col for col in X_data.columns if col.startswith('DRFP_')]
    drfp_data = X_data[drfp_cols].values
    # Filter by variance > 0
    var_mask = drfp_data.var(axis=0) > 0
    return drfp_data[:, var_mask], var_mask

def get_arrhenius_features(X_data):
    """Extract Arrhenius kinetics features"""
    T = X_data['TEMPERATURE'].values
    t = X_data['TIME'].values
    T_kelvin = T + 273.15
    inv_T = 1.0 / T_kelvin
    ln_t = np.log(t + 1e-6)
    interaction = inv_T * ln_t
    return np.column_stack([inv_T, ln_t, interaction, T, t])

def prepare_features(X_data, drfp_mask=None, include_drfp=True):
    """Prepare all features"""
    spange = get_spange_features(X_data)
    arrhenius = get_arrhenius_features(X_data)
    
    if include_drfp:
        drfp_cols = [col for col in X_data.columns if col.startswith('DRFP_')]
        drfp_data = X_data[drfp_cols].values
        if drfp_mask is not None:
            drfp_data = drfp_data[:, drfp_mask]
        features = np.hstack([spange, drfp_data, arrhenius])
    else:
        features = np.hstack([spange, arrhenius])
    
    return features

print("Feature extraction functions defined")

In [ ]:
# MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64], dropout=0.2):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 3))  # 3 outputs
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def train_mlp(X_train, Y_train, input_dim, epochs=200, lr=5e-4, weight_decay=1e-4, hidden_dims=[128, 64]):
    """Train MLP model"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MLPModel(input_dim, hidden_dims=hidden_dims).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
    criterion = nn.HuberLoss()
    
    X_tensor = torch.FloatTensor(X_train).to(device)
    Y_tensor = torch.FloatTensor(Y_train).to(device)
    
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        pred = model(X_tensor)
        loss = criterion(pred, Y_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
    
    return model

print("MLP model defined")

In [ ]:
# Manual OOD Handling Model
class ManualOODModel:
    """
    Model that uses different feature sets for high-error solvents.
    - Full features (Spange + DRFP) for normal solvents
    - Simple features (Spange only) for high-error solvents
    """
    def __init__(self, high_error_solvents, gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3):
        self.high_error_solvents = high_error_solvents
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        
        # Full feature models
        self.scaler_full = StandardScaler()
        self.gp_models_full = []
        self.mlp_models_full = []
        self.lgbm_models_full = []
        
        # Simple feature models (for high-error solvents)
        self.scaler_simple = StandardScaler()
        self.mlp_models_simple = []
        self.lgbm_models_simple = []
        
        self.drfp_mask = None
        self.input_dim_full = None
        self.input_dim_simple = None
    
    def fit(self, X_train, Y_train):
        """Train both full and simple feature models"""
        # Get DRFP mask from training data
        drfp_cols = [col for col in X_train.columns if col.startswith('DRFP_')]
        drfp_data = X_train[drfp_cols].values
        self.drfp_mask = drfp_data.var(axis=0) > 0
        
        # Prepare full features
        X_full = prepare_features(X_train, self.drfp_mask, include_drfp=True)
        self.input_dim_full = X_full.shape[1]
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        
        # Prepare simple features
        X_simple = prepare_features(X_train, None, include_drfp=False)
        self.input_dim_simple = X_simple.shape[1]
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        Y_values = Y_train.values
        
        # Train full feature models
        # GP (only on subset for speed)
        n_gp = min(200, len(X_full_scaled))
        idx_gp = np.random.choice(len(X_full_scaled), n_gp, replace=False)
        for i in range(3):
            kernel = Matern(nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1, n_restarts_optimizer=2)
            gp.fit(X_full_scaled[idx_gp], Y_values[idx_gp, i])
            self.gp_models_full.append(gp)
        
        # MLP (3 models for bagging)
        for _ in range(3):
            mlp = train_mlp(X_full_scaled, Y_values, self.input_dim_full, epochs=200, hidden_dims=[128, 64])
            self.mlp_models_full.append(mlp)
        
        # LightGBM
        lgbm_params = {
            'objective': 'regression',
            'metric': 'mse',
            'learning_rate': 0.03,
            'max_depth': 6,
            'num_leaves': 31,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbose': -1
        }
        for i in range(3):
            model = lgb.LGBMRegressor(**lgbm_params, n_estimators=500)
            model.fit(X_full_scaled, Y_values[:, i])
            self.lgbm_models_full.append(model)
        
        # Train simple feature models
        # MLP (3 models for bagging)
        for _ in range(3):
            mlp = train_mlp(X_simple_scaled, Y_values, self.input_dim_simple, epochs=200, hidden_dims=[64, 32])
            self.mlp_models_simple.append(mlp)
        
        # LightGBM
        for i in range(3):
            model = lgb.LGBMRegressor(**lgbm_params, n_estimators=500)
            model.fit(X_simple_scaled, Y_values[:, i])
            self.lgbm_models_simple.append(model)
        
        return self
    
    def predict(self, X_test):
        """Predict using appropriate model based on solvent"""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Prepare features
        X_full = prepare_features(X_test, self.drfp_mask, include_drfp=True)
        X_full_scaled = self.scaler_full.transform(X_full)
        
        X_simple = prepare_features(X_test, None, include_drfp=False)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # Get predictions from full feature models
        # GP predictions
        gp_preds = np.zeros((len(X_test), 3))
        for i, gp in enumerate(self.gp_models_full):
            gp_preds[:, i] = gp.predict(X_full_scaled)
        gp_preds = np.clip(gp_preds, 0, 1)
        
        # MLP predictions
        mlp_preds_full = []
        X_tensor = torch.FloatTensor(X_full_scaled).to(device)
        for mlp in self.mlp_models_full:
            mlp.eval()
            with torch.no_grad():
                pred = mlp(X_tensor).cpu().numpy()
            mlp_preds_full.append(pred)
        mlp_pred_full = np.mean(mlp_preds_full, axis=0)
        
        # LightGBM predictions
        lgbm_preds_full = np.zeros((len(X_test), 3))
        for i, model in enumerate(self.lgbm_models_full):
            lgbm_preds_full[:, i] = model.predict(X_full_scaled)
        lgbm_preds_full = np.clip(lgbm_preds_full, 0, 1)
        
        # Ensemble full feature predictions
        pred_full = self.gp_weight * gp_preds + self.mlp_weight * mlp_pred_full + self.lgbm_weight * lgbm_preds_full
        
        # Get predictions from simple feature models
        # MLP predictions
        mlp_preds_simple = []
        X_tensor_simple = torch.FloatTensor(X_simple_scaled).to(device)
        for mlp in self.mlp_models_simple:
            mlp.eval()
            with torch.no_grad():
                pred = mlp(X_tensor_simple).cpu().numpy()
            mlp_preds_simple.append(pred)
        mlp_pred_simple = np.mean(mlp_preds_simple, axis=0)
        
        # LightGBM predictions
        lgbm_preds_simple = np.zeros((len(X_test), 3))
        for i, model in enumerate(self.lgbm_models_simple):
            lgbm_preds_simple[:, i] = model.predict(X_simple_scaled)
        lgbm_preds_simple = np.clip(lgbm_preds_simple, 0, 1)
        
        # Ensemble simple feature predictions (no GP, just MLP + LGBM)
        pred_simple = 0.6 * mlp_pred_simple + 0.4 * lgbm_preds_simple
        
        # Select predictions based on solvent
        final_preds = np.zeros((len(X_test), 3))
        solvents = X_test['SOLVENT NAME'].values
        
        for idx in range(len(X_test)):
            solvent = solvents[idx]
            if solvent in self.high_error_solvents:
                final_preds[idx] = pred_simple[idx]
            else:
                final_preds[idx] = pred_full[idx]
        
        return np.clip(final_preds, 0, 1)

print("ManualOODModel defined")

In [ ]:
# Run CV for single solvents
print("Running Single Solvent CV (Leave-One-Out, 24 folds)...")
print("="*60)

# Filter to single solvents only
single_mask = ~X['SOLVENT NAME'].str.contains('\\.')
X_single = X[single_mask].reset_index(drop=True)
Y_single = Y[single_mask].reset_index(drop=True)

print(f"Single solvent samples: {len(X_single)}")
print(f"Unique single solvents: {X_single['SOLVENT NAME'].nunique()}")

# Define high-error solvents for single solvents
high_error_single = [
    '1,1,1,3,3,3-Hexafluoropropan-2-ol',  # HFIP
    '2,2,2-Trifluoroethanol',  # TFE
    'Cyclohexane',
]

# Generate splits
splits = generate_leave_one_out_splits(X_single, Y_single)
print(f"Number of folds: {len(splits)}")

# Track per-solvent errors
solvent_errors = {}
all_preds = []
all_true = []

for fold_idx, (train_idx, test_idx) in enumerate(splits):
    X_train = X_single.iloc[train_idx]
    Y_train = Y_single.iloc[train_idx]
    X_test = X_single.iloc[test_idx]
    Y_test = Y_single.iloc[test_idx]
    
    test_solvent = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = ManualOODModel(high_error_single)
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    solvent_errors[test_solvent] = mse
    
    all_preds.append(preds)
    all_true.append(Y_test.values)
    
    is_high_error = test_solvent in high_error_single
    marker = " [HIGH-ERROR]" if is_high_error else ""
    print(f"Fold {fold_idx+1:2d}: {test_solvent:40s} MSE = {mse:.6f}{marker}")

# Calculate overall MSE
all_preds = np.vstack(all_preds)
all_true = np.vstack(all_true)
single_mse = np.mean((all_preds - all_true) ** 2)
single_std = np.std([solvent_errors[s] for s in solvent_errors])

print(f"\nSingle Solvent CV MSE: {single_mse:.6f} +/- {single_std:.6f}")

In [ ]:
# Analyze per-solvent errors
print("\n" + "="*60)
print("Per-Solvent Error Analysis")
print("="*60)

# Sort by error
sorted_errors = sorted(solvent_errors.items(), key=lambda x: x[1], reverse=True)

print("\nTop 10 highest error solvents:")
for i, (solvent, mse) in enumerate(sorted_errors[:10]):
    is_high_error = solvent in high_error_single
    marker = " [TARGETED]" if is_high_error else ""
    print(f"  {i+1:2d}. {solvent:40s}: {mse:.6f}{marker}")

print("\nTop 10 lowest error solvents:")
for i, (solvent, mse) in enumerate(sorted_errors[-10:]):
    print(f"  {i+1:2d}. {solvent:40s}: {mse:.6f}")

# Compare high-error solvents to baseline
print("\nHigh-error solvent comparison:")
print(f"{'Solvent':<45} {'This Exp':>12} {'Baseline':>12} {'Change':>12}")
print("-"*85)

# Baseline errors from exp_030 (approximate from previous experiments)
baseline_errors = {
    '1,1,1,3,3,3-Hexafluoropropan-2-ol': 0.096,  # HFIP
    '2,2,2-Trifluoroethanol': 0.042,  # TFE
    'Cyclohexane': 0.198,
}

for solvent in high_error_single:
    if solvent in solvent_errors:
        this_exp = solvent_errors[solvent]
        baseline = baseline_errors.get(solvent, 'N/A')
        if isinstance(baseline, float):
            change = (this_exp - baseline) / baseline * 100
            print(f"{solvent:<45} {this_exp:>12.6f} {baseline:>12.6f} {change:>11.1f}%")
        else:
            print(f"{solvent:<45} {this_exp:>12.6f} {baseline:>12}")

print(f"\nMean error for high-error solvents: {np.mean([solvent_errors.get(s, 0) for s in high_error_single if s in solvent_errors]):.6f}")
print(f"Mean error for other solvents: {np.mean([v for k, v in solvent_errors.items() if k not in high_error_single]):.6f}")

In [ ]:
# Run CV for mixtures
print("\n" + "="*60)
print("Running Mixture CV (Leave-One-Ramp-Out, 13 folds)...")
print("="*60)

# Filter to mixtures only
mix_mask = X['SOLVENT NAME'].str.contains('\\.')
X_mix = X[mix_mask].reset_index(drop=True)
Y_mix = Y[mix_mask].reset_index(drop=True)

print(f"Mixture samples: {len(X_mix)}")
print(f"Unique mixtures: {X_mix['SOLVENT NAME'].nunique()}")

# Define high-error solvents for mixtures
high_error_mix = [
    'Acetonitrile.Acetic Acid',
    'Water.Ethanol',
    'Water.2,2,2-Trifluoroethanol',
]

# Generate splits
mix_splits = generate_leave_one_ramp_out_splits(X_mix, Y_mix)
print(f"Number of folds: {len(mix_splits)}")

# Track per-mixture errors
mix_errors = {}
mix_preds = []
mix_true = []

for fold_idx, (train_idx, test_idx) in enumerate(mix_splits):
    X_train = X_mix.iloc[train_idx]
    Y_train = Y_mix.iloc[train_idx]
    X_test = X_mix.iloc[test_idx]
    Y_test = Y_mix.iloc[test_idx]
    
    test_mixture = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = ManualOODModel(high_error_mix)
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    mix_errors[test_mixture] = mse
    
    mix_preds.append(preds)
    mix_true.append(Y_test.values)
    
    is_high_error = test_mixture in high_error_mix
    marker = " [HIGH-ERROR]" if is_high_error else ""
    print(f"Fold {fold_idx+1:2d}: {test_mixture:40s} MSE = {mse:.6f}{marker}")

# Calculate overall MSE
mix_preds = np.vstack(mix_preds)
mix_true = np.vstack(mix_true)
mix_mse = np.mean((mix_preds - mix_true) ** 2)
mix_std = np.std([mix_errors[s] for s in mix_errors])

print(f"\nMixture CV MSE: {mix_mse:.6f} +/- {mix_std:.6f}")

In [ ]:
# Calculate overall CV score
print("\n" + "="*60)
print("Overall Results")
print("="*60)

# Weighted average (same as competition)
n_single = len(all_true)
n_mix = len(mix_true)
n_total = n_single + n_mix

overall_mse = (n_single * single_mse + n_mix * mix_mse) / n_total

print(f"\nSingle Solvent CV MSE: {single_mse:.6f} +/- {single_std:.6f} (n={n_single})")
print(f"Mixture CV MSE: {mix_mse:.6f} +/- {mix_std:.6f} (n={n_mix})")
print(f"Overall CV MSE: {overall_mse:.6f}")

print(f"\nBaseline (exp_030): CV = 0.008298")
print(f"Improvement: {(0.008298 - overall_mse) / 0.008298 * 100:.1f}%")

# Check if this is better than baseline
if overall_mse < 0.008298:
    print("\n✓ BETTER than baseline! Consider submitting.")
else:
    print("\n✗ WORSE than baseline. Need to adjust approach.")

In [ ]:
# Summary and next steps
print("\n" + "="*60)
print("Summary of Manual OOD Handling Experiment")
print("="*60)

print(f"\nManual OOD Handling CV MSE: {overall_mse:.6f}")
print(f"Baseline (exp_030): CV = 0.008298")
print(f"Improvement: {(0.008298 - overall_mse) / 0.008298 * 100:.1f}%")

print("\nPer-solvent comparison for high-error solvents:")
for solvent in high_error_single:
    if solvent in solvent_errors:
        this_exp = solvent_errors[solvent]
        baseline = baseline_errors.get(solvent, 'N/A')
        if isinstance(baseline, float):
            change = (this_exp - baseline) / baseline * 100
            print(f"  {solvent}: {this_exp:.6f} (was {baseline:.6f}, {change:+.1f}%)")

print("\nKey Insights:")
print("1. Manual OOD handling targets specific high-error solvents")
print("2. Uses simpler features (Spange only) for these solvents")
print("3. Uses full features (Spange + DRFP) for other solvents")

if overall_mse < 0.008298:
    print("\nCONCLUSION: Manual OOD handling IMPROVES overall CV.")
    print("Consider submitting to test if this changes the CV-LB relationship.")
else:
    print("\nCONCLUSION: Manual OOD handling does NOT improve overall CV.")
    print("The improvement on high-error solvents doesn't compensate for other solvents.")
    print("\nNext steps:")
    print("1. Try mixall-style ensemble (MLP + XGBoost + RF + LightGBM)")
    print("2. Try ensemble disagreement for OOD detection")
    print("3. Consider different high-error solvent list")